In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [118]:
df = pd.read_csv('data.csv')

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X = df.drop('overall_rating', axis=1)
y = df['overall_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

In [120]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42, n_jobs=-1 , n_estimators=122 , max_depth=25 , min_samples_split=3,min_samples_leaf = 3, bootstrap=True, max_features = 'sqrt' )
rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)

y_pred = rf.predict(X_test)

y_data = rf.predict(X)

print('Train Set:')
print(f'MSE: {mean_squared_error(y_train, y_train_pred)}')
print(f'MAE: {mean_absolute_error(y_train, y_train_pred)}')
print(f'R2: {r2_score(y_train, y_train_pred)}')

print()

print('Test Set:')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'R2: {r2_score(y_test, y_pred)}')

print()

print('Data Set:')
print(f'MSE: {mean_squared_error(y, y_data)}')
print(f'MAE: {mean_absolute_error(y, y_data)}')
print(f'R2: {r2_score(y, y_data)}')

Train Set:
MSE: 1.187132047905243
MAE: 0.6863472766979463
R2: 0.9751626576310508

Test Set:
MSE: 3.160691268348987
MAE: 1.1840973686730287
R2: 0.9354730626093268

Data Set:
MSE: 1.8450308183212984
MAE: 0.8522754992172865
R2: 0.9617165809426073
